<a href="https://colab.research.google.com/github/elfavci/Cifar_10_Classification/blob/main/Cifar_10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Cifar-10 Image Classification**

In [ ]:
import numpy as np
import pandas as pd
import cv2

In [ ]:
from sklearn.datasets import fetch_openml
from skimage.filters import threshold_otsu
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split  #veri kümesini böler

Cifar-10 Veri kumesini ekleme

In [ ]:
from tensorflow.keras.datasets import cifar10

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
for i in range(5):
    plt.imshow(x_test[i])
    plt.title("Class:"+str(y_test[i][0]))
    plt.axis("off")
    plt.show()

Veri setinden ornek goruntu ve cikti elde etme

In [ ]:
plt.imshow(x_train[1])
plt.axis("off")
print("y= ",y_train[1])

Egitim ve test verilerinin birlestirilmesi

In [ ]:
x_data=np.concatenate((x_train,x_test),axis=0)
y_data=np.concatenate((y_train,y_test),axis=0)

In [ ]:
len(x_data)

In [ ]:
len(y_data)

In [ ]:
x_data[100]

In [ ]:
y_data[100]

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2,random_state=42)

In [ ]:
len(x_train)

In [ ]:
x_train[0]

In [ ]:
y_train[0]

In [ ]:
#verileri 0-1 arasında normalize edilmesi
x_train=x_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0

In [ ]:
x_train[100]

In [ ]:
#one hot encoding yöntemi:
num_classes=10
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [ ]:
y_train[0]

# ***Convolutional Neural Network (CNN)***

* Sequential: Katmanları sırayla eklemek için kullanılır. Yani modelin katmanlarını tek tek üst üste ekle
* Conv2D: Görüntüler üzerinde 2D konvolüsyon işlemi yapar. Görselden anlamlı özellikler çıkarmak için kullanılır.
* MaxPooling2D: Boyutları küçültür ve önemli özellikleri öne çıkarır. Bu işlem, modelin daha az hesaplama yapmasını sağlar.
* Flatten: Çok boyutlu verileri tek bir boyuta çevirir. Fully connected katmanlara geçişte kullanılır.
* Dense: Tam bağlantılı sinir ağı katmanıdır, sınıflandırma işlemleri burada yapılır.
* Dropout: Overfitting’i önlemek için bazı bağlantıları rastgele kapatır.

Gerekli Kütüphaneler:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Input

In [ ]:
model=Sequential()

**model.add(Conv2D(32,(3,3),activation="relu",padding="same",input_shape=x_train.shape[1:]))**
* padding="same": Çıktının boyutlarını girişle aynı tutar. Yani, kenarlara sıfırlar eklenir (zero-padding) ve çıktı boyutları girişle aynı kalır.
* input_shape=x_train.shape[1:]: Giriş boyutunu dinamik olarak belirler. Örneğin, x_train.shape[1:] CIFAR-10 için (32, 32, 3) döndürür. Bu, giriş şeklinin manuel olarak belirtilmesini önler ve daha esnek bir yapı sağlar.
* relu: Aktivasyon fonksiyonu, negatif değerleri sıfıra dönüştürür, öğrenmeyi hızlandırır.
* 32 adet 3x3 filtre uygular.
* Görselden kenar, köşe gibi düşük seviyeli özellikleri çıkarır.

**model.add(MaxPooling2D(pool_size=(2,2))) **
* 2x2 boyutunda havuzlama yaparak, boyutu yarıya indirir.
* Önemli bilgileri kaybetmeden boyut küçültülür.

In [ ]:
model.add(Input(shape=x_train.shape[1:]))  # Giriş katmanı
model.add(Conv2D(32,(3,3),activation="relu",padding="same"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),activation="relu"))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation="softmax"))

In [ ]:
#modelin derlenmesi:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
#modeli eğitme:
batch_size=128
epochs=30
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test))

In [ ]:
scores=model.evaluate(x_test,y_test,verbose=1)
print("test loss: ",scores[0])
print("test accuracy: ",scores[1])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [ ]:
# Model ile tahmin yap
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # En yüksek olasılığı olan sınıfı seç


In [ ]:
# y_test'i düz etiket formatına dönüştür (eğer one-hot encoded ise)
if len(y_test.shape) > 1:
    y_test_classes = np.argmax(y_test, axis=1)
else:
    y_test_classes = y_test



In [ ]:
# Sınıf bazında metrikleri hesapla
print("Classification Report:\n")
print(classification_report(y_test_classes, y_pred_classes, target_names=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck']))


In [ ]:
# Confusion Matrix hesapla
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Görselleştirme
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'],
    yticklabels=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'])
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.title('Confusion Matrix')
plt.show()


# ***Convolutional Neural Network 2 (CNN)***

In [ ]:
modelT=Sequential()

In [ ]:
modelT.add(Input(shape=x_train.shape[1:]))  # Giriş katmanı
modelT.add(Conv2D(32,(3,3),activation="relu",padding="same"))
modelT.add(MaxPooling2D(pool_size=(2,2)))
modelT.add(Conv2D(64,(3,3),activation="relu"))
modelT.add(MaxPooling2D(pool_size=(2,2)))
modelT.add(Conv2D(128,(3,3),activation="relu"))
modelT.add(Conv2D(256,(3,3),activation="relu"))
modelT.add(Flatten())
modelT.add(Dense(512,activation="relu"))
modelT.add(Dropout(0.5))
modelT.add(Dense(num_classes,activation="softmax"))

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
#modelin derlenmesi:
modelT.compile(loss='categorical_crossentropy',optimizer="adam",metrics=["accuracy"])


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#  Veri Artırma (Augmentation)
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
datagen.fit(x_train)

🔍 Batch Size Nedir ve Ne İşe Yarar?
Batch size, her eğitim adımında (iteration) modele kaç örnek sunulacağını belirler. Yani, model ağırlıklarını güncellemeden önce kaç örnekle öğrenim yapacağını tanımlar.

🎯 Ne Zaman Hangi Batch Size Kullanılır?
Küçük Veri Setleri: Küçük batch size (16-32) genellikle daha iyi sonuç verir.
Orta Büyüklükteki Veri Setleri: (64-128) çoğu zaman en iyi dengeyi sağlar.
Büyük Veri Setleri ve Güçlü GPU: Büyük batch size (256-512) daha hızlı eğitim sağlar.

In [ ]:
#modeli eğitme:
batch_size=128
epochs=20
modelT.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test))

In [ ]:
scores=modelT.evaluate(x_test,y_test,verbose=1)
print("test loss: ",scores[0])
print("test accuracy: ",scores[1])

In [ ]:
# Model ile tahmin yap
y_predx = modelT.predict(x_test)
y_pred_class = np.argmax(y_predx, axis=1)  # En yüksek olasılığı olan sınıfı seç

In [ ]:
# y_test'i düz etiket formatına dönüştür (eğer one-hot encoded ise)
if len(y_test.shape) > 1:
    y_test_class = np.argmax(y_test, axis=1)
else:
    y_test_class = y_test



In [ ]:
# Sınıf bazında metrikleri hesapla
print("Classification Report:\n")
print(classification_report(y_test_class, y_pred_class, target_names=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck']))


In [ ]:
# Confusion Matrix hesapla
cmm = confusion_matrix(y_test_class, y_pred_class)

# Görselleştirme
plt.figure(figsize=(10, 8))
sns.heatmap(cmm, annot=True, fmt='d', cmap='Blues', xticklabels=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'],
    yticklabels=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'])
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.title('Confusion Matrix')
plt.show()


# CNN model 3


In [ ]:
modelx=Sequential()

In [ ]:
from tensorflow.keras.layers import BatchNormalization

In [ ]:
modelx.add(Input(shape=x_train.shape[1:]))  # Giriş katmanı
modelx.add(Conv2D(32,(3,3),activation="relu",padding="same"))
modelx.add(BatchNormalization())
modelx.add(MaxPooling2D(pool_size=(2,2)))
modelx.add(Conv2D(64,(3,3),activation="relu"))
modelx.add(Dropout(0.3))  # Conv2D katmanlarından sonra
modelx.add(BatchNormalization())
modelx.add(MaxPooling2D(pool_size=(2,2)))
modelx.add(Conv2D(128,(3,3),activation="relu"))
modelx.add(Dropout(0.3))  # Conv2D katmanlarından sonra
modelx.add(BatchNormalization())
modelx.add(Conv2D(256,(3,3),activation="relu"))
modelx.add(BatchNormalization())
modelx.add(MaxPooling2D(pool_size=(2,2)))
modelx.add(Conv2D(512, (3, 3), activation="relu", padding="same"))
modelx.add(Dropout(0.3))  # Conv2D katmanlarından sonra
modelx.add(BatchNormalization())
modelx.add(Flatten())
modelx.add(Dropout(0.5))  # Dense katmandan önce
modelx.add(Dense(512,activation="relu"))
modelx.add(Dropout(0.5))
modelx.add(Dense(num_classes,activation="softmax"))

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)
modelx.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

modelx.compile(loss='categorical_crossentropy',optimizer="adam",metrics=["accuracy"])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datage = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)



In [ ]:
#  Veri Artırma (Augmentation)

datag = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
datag.fit(x_train)

In [ ]:
#modeli eğitme:
batch_size=128
epochs=50
modelx.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(x_test,y_test))

In [ ]:
scores=modelx.evaluate(x_test,y_test,verbose=1)
print("test loss: ",scores[0])
print("test accuracy: ",scores[1])

In [ ]:
# Model ile tahmin yap
y_predxi = modelx.predict(x_test)
y_pred_classx = np.argmax(y_predxi, axis=1)  # En yüksek olasılığı olan sınıfı seç

In [ ]:
# y_test'i düz etiket formatına dönüştür (eğer one-hot encoded ise)
if len(y_test.shape) > 1:
    y_test_clas = np.argmax(y_test, axis=1)
else:
    y_test_clas = y_test



In [ ]:
# Sınıf bazında metrikleri hesapla
print("Classification Report:\n")
print(classification_report(y_test_clas, y_pred_classx, target_names=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck']))


In [ ]:
# Confusion Matrix hesapla
cmn = confusion_matrix(y_test_clas, y_pred_classx)

# Görselleştirme
plt.figure(figsize=(10, 8))
sns.heatmap(cmn, annot=True, fmt='d', cmap='Greens', xticklabels=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'],
    yticklabels=[
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'])
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.title('Confusion Matrix')
plt.show()


# RESNET50
ResNet50, derin sinir ağlarında katman sayısının artmasıyla ortaya çıkan vanishing gradient sorununu çözmek için geliştirilmiş bir arttırılmış (residual) ağ yapısıdır. Keras'ta hazır olarak gelir ve transfer learning (aktarım öğrenmesi) ile çok güçlü sonuçlar verir.